# Decision Tree Implimentation from Scratch
### Uses Information Gain as measurment split and using StoppingEarly

In [601]:
# Importing all required libraries
import numpy as np
import pandas as pd
from sklearn import datasets

In [602]:
# Loading datasets
iris=datasets.load_iris()

In [603]:
# Converting the bunch type into df for easy visualisation + all are in int
iris_df=pd.DataFrame(data=np.array(iris.data,dtype=int),columns=iris.feature_names)
iris_df['target']=iris.target #adding a new column named target to store the targetDatapoints 
print(iris_df)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                    5                 3                  1                 0   
1                    4                 3                  1                 0   
2                    4                 3                  1                 0   
3                    4                 3                  1                 0   
4                    5                 3                  1                 0   
..                 ...               ...                ...               ...   
145                  6                 3                  5                 2   
146                  6                 2                  5                 1   
147                  6                 3                  5                 2   
148                  6                 3                  5                 2   
149                  5                 3                  5                 1   

     target  
0         0  

In [604]:
# Checking if there is any empty or nan slot there in df
print(iris_df.isna()) # will give false for nonempty slots
print(iris_df.isna().sum()) # will give summation of all empty slots in each column

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                False             False              False             False   
1                False             False              False             False   
2                False             False              False             False   
3                False             False              False             False   
4                False             False              False             False   
..                 ...               ...                ...               ...   
145              False             False              False             False   
146              False             False              False             False   
147              False             False              False             False   
148              False             False              False             False   
149              False             False              False             False   

     target  
0     False  

In [605]:
names=iris.target_names
print(names)

['setosa' 'versicolor' 'virginica']


# Defining Entropy 

In [606]:
# Defniing Entropy
def entropy(col):
    counts = np.unique(col,return_counts=True) #Returns a tupple with unique digits array and another array for there frequency
    N = float(col.shape[0]) #.shape returns tupple, for 1d: containing a shape-number and dimension else return (row x col)
    ent = 0.0
    
    for i in counts[1]: # In counts tupple, 1st array would be required as it contained frequencies of their respective "index" for calculating the probability 
        p  = i/N
        ent += (-1.0*p*np.log2(p)) #np.log2() used for calculating logbase2 of array and scalers
        # summing up for getting the entropy    
    return ent

In [607]:
# Above operations's example:

col1d=np.array([11,12,11,34,45,34,23,23])

# 1)
count1=np.unique(col1d,return_counts=True)
print(type(count1))
print(count1)
print(count[1])

print()

# 2)
N = col1d.shape
print(N, N[0]) #for array1d, returns tupple of 2 elements: no. of elements and comma empty space represents a 1d array
print(type(N))
col2d=np.array([[1,2,4],[5,6,7]])
print(col2d.shape) #their product give the actual shape (rows x col)

<class 'tuple'>
(array([11, 12, 23, 34, 45]), array([2, 1, 2, 2, 1], dtype=int64))
[2 1 2 2 1]

(8,) 8
<class 'tuple'>
(2, 3)


# Defining the code for division of node dataFrame into right and left node

In [608]:
# This Python function, divide_data, appears to be designed to split a Pandas DataFrame x_data into two separate DataFrames based on a specified key and a threshold value. The function takes three parameters:
# x_data: The Pandas DataFrame that you want to split.
# key: The column name in the DataFrame based on which the splitting will be performed.
# val: The threshold value for the specified key column.
def divide_data(x_data, key, val):
    # Create empty DataFrames for the left and right subsets
    x_right = pd.DataFrame([], columns=x_data.columns) #Represents Empty dataFrame
    x_left = pd.DataFrame([], columns=x_data.columns)

    # Iterate over each row in the original DataFrame
    for i in range(x_data.shape[0]):
        # Extract the value from the key column for the current row
        val_ = x_data[key].loc[i]

        # Compare the value with the specified threshold
        if val_ > val:
            # If the value is greater than the threshold, append the row to the right subset
            # x_right = pd.concat([x_right, x_data.loc[i]])
            x_right = x_right.append(x_data.loc[i], ignore_index=True)
        else:
            # If the value is less than or equal to the threshold, append the row to the left subset
            # x_left = pd.concat([x_left, x_data.loc[i]])
            x_left = x_left.append(x_data.loc[i], ignore_index=True)
    # Return the two subsets
    return x_left, x_right

In [609]:
# Create a sample DataFrame with personalised column and row
data1 = {'A': [1, 2, 3],
        'B': [4, 5, 6],
        'C': [7, 8, 9]}
index_labels1 = ['row1', 'row2', 'row3'] 
df = pd.DataFrame(data, index=index_labels1) #index used for assining rows
df

,A,B,C
row1,1,4,7
row2,2,5,8
row3,3,6,9


# Defining Info Gain

In [610]:
def information_gain(x_data, key, val):
    # Split the data into left and right subsets based on the key and threshold
    left, right = divide_data(x_data, key, val)
    
    # Calculate the number of samples in Left and Right / total no. of samples
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    # Check if all examples go to one side (no split)
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -10000000  # Minimum Information Gain (a large negative value)
    
    # Calculate information gain using the entropy function
    i_gain = entropy(x_data.target) - (l * entropy(left.target) + r * entropy(right.target))   
    
    # Return the calculated information gain
    return i_gain


## Defining the function in which it would store the output of counts of labels of outputs

In [611]:
def find_count(X_train):
    # Initialize an empty list to store counts of each output in correspoding indexs
    count = []
    
    # Count occurrences of class label 0 and append to the list in 0th index
    count.append(X_train[X_train.target == 0].shape[0])
    
    # Count occurrences of class label 1 and append to the list in 1st index
    count.append(X_train[X_train.target == 1].shape[0])
    
    # Count occurrences of class label 2 and append to the list in 2nd index
    count.append(X_train[X_train.target == 2].shape[0])
    
    # Return the list containing counts of each class
    return count


In [612]:
#Classes and objects with constructor
class MyClass:
    def __init__(self, parameter1, parameter2):
        self.attribute1 = parameter1
        self.attribute2 = parameter2


In [613]:
# Creating an instance of MyClass
my_instance = MyClass("value3", "value2")

# Accessing attributes of the instance
print(my_instance.attribute1)  # Output: value1
print(my_instance.attribute2)  # Output: value2


value3
value2


## Now defining the "Training-function"

In [614]:
class DecisionTree:
    
    #Constructor
    def __init__(self,depth=0,max_depth=5): #self : indicates the object created from this class can access the attribute else will not able to
        self.left = None
        self.right = None
        self.key = None
        self.val = None
        self.count = None
        self.max_depth = max_depth #for avoiding overfitting and complex tree, intiliased maximun depth
        self.depth = depth #done because by default building of tree should start from depth:0, represents the root node's level, goes till max_depth which is termination of the tree's growth for avoiding overfitting
        self.target = None
        #And its not compulsory to write all attributes of the object which are being used and altered as tree grows, to NONE at constructor as if not defined still thie default values would be NONE, but it shows the "GOOD-PRACTISE" as defined the intial stage of the attributes which are being used and altered
        
    def train(self, X_train, names):  # name represents the className or labelName
        features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
        info_gains = []
        
        # Calculate information gain for each feature
        for i in features:
            i_gain = information_gain(X_train, i, X_train[i].mean())  # Threshold value in infoGain is the mean of all datapoints of the respective features
            info_gains.append(i_gain)
        
        # Determine the feature to split on based on maximum information gain
        self.key = features[np.argmax(info_gains)]  # returns the maximum value index of the array list
        self.val = X_train[self.key].mean()  # it would take the mean value of the feature having the maximum infoGain()
        
        # Print information about the current node
        print("Level", self.depth)
        self.count = find_count(X_train)  # take the number of all respective output labels 
        cnt = 0  # Maintain to check if there is only 1 flower in this node, so that could get the leaf node
        
        # Display counts for each class in the current node
        for i in range(len(self.count)):
            if self.count[i]:
                print("Count of", names[i], "=", self.count[i])
                cnt += 1
        
        print("Current entropy =", entropy(X_train.target))
        
        # If there is more than one flower in the node, print information about splitting
        if cnt != 1:
            print("Splitting on Tree Features", self.key, "with information gain", np.argmax(info_gains))
        
        # Split Data
        data_left, data_right = divide_data(X_train, self.key, self.val)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        # reset_index(drop=True), it again resets the row/index to 0 of the child/split DataFrame, in which it could be treated as an individual/separate DataFrame 
        
        # If it's truly a leaf node, means that only one type of flower is there in the targetColumn
        if cnt == 1:
            # Assign the target class based on the mean of the "target" column
            if X_train.target.mean() >= 1.5:
                self.target = names[2]
            elif X_train.target.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Reached leaf Node")
            print()
            print()
            return # As leaf reached , return makes program counter to leave/exit the training of the current node
        
        # Stop early when the depth >= max depth
        if self.depth >= self.max_depth:
            # Assign the target class based on the mean of the "Output" column
            if X_train.target.mean() >= 1.5:
                self.target = names[2]
            elif X_train.target.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Max depth Reached")
            print()
            print()
            return # As max depth reached , return makes program counter to leave/exit the training of the current node
        
        print()
        print()
        
        # Recursive Case: Train left and right child nodes
        self.left = DecisionTree(depth=self.depth+1, max_depth=self.max_depth) # made the object/instance for the child
        self.left.train(data_left, names) #called the object's function of the class
        
        self.right = DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
        self.right.train(data_right, names)
        
        # # Set the target at every node based on the mean of the "Output" column
        # if X_train.target.mean() >= 1.5:
        #     self.target = names[2]
        # elif X_train.target.mean() <= 0.5:
        #     self.target = names[0]
        # else:
        #     self.target = names[1]
        
        return

In [615]:
# #About np.argmax() and np.max()
# list=[1,2,3,4]
# print(np.argmax(list)) #return the index having maxValue
# np.max(list) #return the maxValue

In [616]:
# # reset_index(drop=True), it again reset the row/index to 0 of the child/splitted dataFrame, in which it could treated as individual/separete DataFrame 
# data_df = pd.DataFrame({'A':[1,2,3,4],'B':[5,6,7,8]})
# data_left = data_df[data_df['A'] < 3]
# data_right = data_df[data_df['A'] >= 3]
# print(data_left)
# print()
# print(data_right)
# print()
# data_left = data_left.reset_index(drop=True)  # Now, data_left has index/row [0, 1]
# data_right = data_right.reset_index(drop=True)  # Now, data_right has index/row [0, 1]
# print(data_left)
# print()
# print(data_right)

In [617]:
# # Example data
# output_list = [ 2, 2,2,2, 2, 2]

# # Calculate mean of the output list
# mean_output = np.mean(output_list)

# # Assign target class based on mean
# if mean_output < 1:
#     target = 0
# elif mean_output < 2:
#      target = 1
# else:
#      target = 1

# print(f"Mean Output: {mean_output}, Target Class: {target}")


## Making the object/instance 

In [618]:
df=DecisionTree()

In [619]:
df.train(iris_df,names)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_

Level 0
Count of setosa = 50
Count of versicolor = 50
Count of virginica = 50
Current entropy = 1.584962500721156
Splitting on Tree Features petal width (cm) with information gain 3


C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)


C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_

Level 1
Count of setosa = 50
Current entropy = 0.0
Reached leaf Node




...

C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(



Level 5
Count of virginica = 3
Current entropy = 0.0
Reached leaf Node


Level 5
Count of versicolor = 2
Count of virginica = 9
Current entropy = 0.6840384356390417
Splitting on Tree Features sepal width (cm) with information gain 1
Max depth Reached


Level 4
Count of virginica = 4
Current entropy = 0.0
Reached leaf Node




C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_

Level 3
Count of virginica = 28
Current entropy = 0.0
Reached leaf Node




C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_right = x_right.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_data.loc[i], ignore_index=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_8024\733337414.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_left = x_left.append(x_da